In [1]:
import numpy as np
from scipy.special import expit

np.random.seed(123)
n = 100000
p = 100
x = np.random.normal(size=(n, p))
beta = np.random.normal(size=p)
prob = expit(-0.5 + x.dot(beta))  # p = 1 / (1 + exp(-x * beta))
y = np.random.binomial(1, prob, size=n)
dat = np.hstack((y.reshape(n, 1), x))
np.savetxt("logistic_data.txt", dat, fmt="%.8f", delimiter="\t")

In [2]:
import findspark
findspark.init("/Users/xinby/Library/Spark")
from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.\
    master("local[4]").\
    appName("PySpark RDD").\
    getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")
print(spark)
print(sc)

23/05/04 15:15:22 WARN Utils: Your hostname, XinBys-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.65.116.79 instead (on interface en0)
23/05/04 15:15:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/04 15:15:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
<SparkContext master=local[4] appName=PySpark RDD>


In [3]:
# load softplus
test = False
def softplus(x):
    # 此处插入代码
    if (test): 
        print(f"call func: softplus")

    ans = np.where(x>=0,np.log(1+np.exp(-x))+x,np.log(1+np.exp(x)))
    
    if (test): 
        print(f"return.shape:{ans.shape}")
        print(f"end func: softplus \n")
        
    return ans

# load data to rdd
def string_to_vector(line):
    vector = line.split("\t")
    return np.array(vector, dtype=float)

def partition_to_matrix (iterator):
    iterator_vec = map(string_to_vector, iterator)
    data = list(iterator_vec)
    if len(data) < 1:
        matrix = np.array([])
    else:
        matrix = np.vstack(data)
    yield matrix

file = sc.textFile("logistic_data.txt")

data_partition = file.mapPartitions(partition_to_matrix)
data_partition_nonempty = data_partition.filter(lambda x: x.shape[0] > 0)
data_partition_nonempty.cache()
data_partition_nonempty.count()



4

In [4]:
# compute beta_hat fcn
def compute_betahat(mat,beta_old):

    if (test):
        print("call fnc: comp_bhat\n")

    y = mat[:,0].
    if (test):
        print(f"0y{y.shape}\n")

    x = mat
    x[:,0]=1

    if (test):
        print(f"0x{x.shape}\n")

    xbeta = x.dot(beta_old)
    if (test):
        print(f"1xb{xbeta.shape}\n")

    prob = expit(xbeta) #这里的prob可以避免吗sigmoid（因为后面的objfn只用了softplus）？但是W该怎么算啊
    if (test):
        print(f"2pr{prob.shape}\n")

    w = prob * (1.0 - prob) + 1e-6
    if (test):
        print(f"3w{w.shape}\n")

    xtw = x.transpose() * w
    if (test):
        print(f"4xtw{xtw.shape}\n")

    xtwx = xtw.dot(x)
    if (test): 
        print(f"5xtwx{xtwx.shape}\n")

    z = xbeta + (y - prob) / w
    if (test): 
        print(f"6z{z.shape}\n")

    xtwz = xtw.dot(z)
    if (test): 
        print(f"7xtwz{xtwz.shape}\n")
        print(f"cal objfn = -np.sum(y*(x-softplus(x))+(y-1)*softplus(x))")
        print(f"y*xbeta{(y*xbeta).shape}")
    
    objfn = -np.sum( y * (xbeta-softplus(xbeta) ) + (y-1) * softplus(xbeta) )
    
    if (test): 
        print(f"8objfn{objfn}\n")

    if (test): 
        print("return from comp_beta")

    return xtwx, xtwz, objfn

# iter computation

p = data_partition_nonempty.first().shape[1]-1 #subtract y
beta_hat = np.zeros(p+1)#initialization
object_values = [] #init

MaxIteration = 100 #iter settings
epsilon = 1e-6 #iter settings

for i in range(MaxIteration):
    if (test):
        print(f"start iter:{i}")
    # 完整数据的 X'WX 和 X'Wz 是各分区的加和
    xtwx, xtwz, objfn = data_partition_nonempty.\
        map(lambda part: compute_betahat(part, beta_hat)).\
        reduce(lambda x, y: (x[0] + y[0], x[1] + y[1], x[2] + y[2]))
    # 计算新 beta
    beta_new = np.linalg.solve(xtwx, xtwz)
    if (test):
        print(f"bn{beta_new.shape}")
    # 计算 beta 的变化
    resid = np.linalg.norm(beta_new - beta_hat)
    if (np.mod(i,5)==0 or i==MaxIteration-1):
        print(f"Iteration {i}, objfn = {objfn}, resid = {resid}")
    object_values.append(objfn)
    # 如果 beta 几乎不再变化，退出循环
    if resid < epsilon:
        print(f"Iteration {i}, objfn = {objfn}, resid = {resid}")
        break
    # 更新 beta
    beta_hat = beta_new

    ############Q: 这里是说要自己推一个Gradient的递推公式？

Iteration 0, objfn = 69314.71805599453, resid = 1.9999920000320117
Iteration 5, objfn = 202.86044631969702, resid = 1.0015354523470537
Iteration 10, objfn = 1.421137445056786, resid = 0.9342716889375833
Iteration 15, objfn = 0.04573482357272951, resid = 0.31382220017651186
Iteration 20, objfn = 0.015281002951894607, resid = 0.13255438286225463
Iteration 25, objfn = 0.008887144886671194, resid = 0.08161794162574054
Iteration 30, objfn = 0.006224562554280055, resid = 0.05859814575533662
Iteration 35, objfn = 0.004778199880206557, resid = 0.04560299425169134
Iteration 40, objfn = 0.0038729130125148004, resid = 0.03728511111661348
Iteration 45, objfn = 0.0032540171446271415, resid = 0.03151467802325314
Iteration 50, objfn = 0.0028046208910836867, resid = 0.027281077801450948
Iteration 55, objfn = 0.0024636911177822185, resid = 0.024044526487084994
Iteration 60, objfn = 0.0021962975438327703, resid = 0.021490968531090004
Iteration 65, objfn = 0.00198102654280774, resid = 0.01942544205586571

In [5]:
x = dat
x[:,0] = 1
print(beta_hat)
xbetahat = x.dot(beta_hat)
probhat = expit(xbetahat)
print(probhat)

[ 1.82606808e+01 -6.42966967e-15  2.50757163e-16 -1.13096510e-14
  1.13611481e-14  8.79546633e-15 -1.36304139e-14 -1.31772968e-14
 -6.26401961e-19 -2.90420119e-15 -7.51085147e-17 -3.31720583e-15
 -8.93853355e-17 -9.87538988e-15  7.94735760e-15 -4.36984040e-15
 -1.61157090e-14 -2.02442529e-15  1.03671228e-14 -4.12070588e-15
  2.24014708e-14 -1.08807404e-14 -4.47106877e-15  1.91860400e-14
  1.01117976e-14 -1.20879950e-14  6.71999571e-15 -1.08185291e-14
 -1.20263466e-16  1.38023847e-15 -7.32651923e-15  1.26341588e-14
  6.23554394e-15 -1.78793313e-14 -6.93090602e-15  8.10153345e-15
  2.67888769e-14  1.22746367e-14  2.02740305e-14 -4.31849411e-16
  1.23355130e-16  1.49628461e-14  4.04542951e-16 -1.28348703e-14
 -2.16672095e-17  8.52851970e-15  4.75046170e-15  5.40555375e-15
 -1.38173165e-14  1.58867995e-15  1.42244199e-14 -1.80736878e-14
  2.64692890e-16 -1.11307536e-14 -1.06756611e-16  1.14124746e-15
 -1.35398306e-14 -8.28340179e-15 -3.44426206e-15 -2.53127386e-14
  2.44055200e-16 -1.30679